## Use albumentations library for image augmentation
To generate object detection dataset. Currently works for Pascal VOC format

Notebook author: yptheangel(Choo Wilson)

References:<br> 
https://www.curiousily.com/posts/image-data-augmentation-for-tensorflow-2-keras-and-pytorch-with-albumentations-in-python/
https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/example_bboxes.ipynb

In [2]:
%matplotlib inline
import cv2
import os
import os.path as osp
import xml.etree.ElementTree as ET
import numpy as np
import albumentations as A
from albumentations import (Compose,BboxParams)
from matplotlib import pyplot as plt

In [3]:
def getObjects(fullname):
    CLS=['avocado','banana']
    bb = ""
    objs=[]
    cls_ids=[]
    in_file = open(fullname)
    tree=ET.parse(in_file)
    root = tree.getroot()
    # iterate through the objects inside a xml file
    for i, obj in enumerate(root.iter('object')):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in CLS or int(difficult)==1:
            continue
        cls_id = CLS.index(cls)
        xmlbox = obj.find('bndbox')
        b = [int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text)]
        c_id = cls_id
        objs.append(b)
        cls_ids.append(c_id)
    return objs,cls_ids

In [4]:
# Functions to visualize bounding boxes and class labels on an image. 
# Based on https://github.com/facebookresearch/Detectron/blob/master/detectron/utils/vis.py

BOX_COLOR = (0, 200,0)
TEXT_COLOR = (0, 0, 0)

def visualize_bbox(img, bbox, class_id, class_idx_to_name, color=BOX_COLOR, thickness=2):
#     x_min, y_min, w, h = bbox
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color=color, thickness=thickness)
    class_name = class_idx_to_name[class_id]
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (int(x_min), int(y_min - int(1.3 * text_height))), (int(x_min + text_width), int(y_min)), BOX_COLOR, -1)
    cv2.putText(img, class_name, (int(x_min), int(y_min - int(0.3 * text_height))), cv2.FONT_HERSHEY_SIMPLEX, 0.35,TEXT_COLOR, lineType=cv2.LINE_AA)
    return img

def visualize(annotations, category_id_to_name):
    img = annotations['image'].copy()
    for idx, bbox in enumerate(annotations['bboxes']):
        img = visualize_bbox(img, bbox, annotations['category_id'][idx], category_id_to_name)
    plt.figure(figsize=(18, 32))
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [5]:
def get_aug(aug, min_area=0., min_visibility=0.):
       return Compose(aug, bbox_params=BboxParams(format='pascal_voc', min_area=min_area, min_visibility=min_visibility, label_fields=['category_id']))

In [6]:
doc_aug = [
    A.RandomCrop(p=0.5,height=200,width=200),
    A.RandomBrightnessContrast(p=0.6),
    A.Rotate(limit=10, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.2),
]

In [7]:
category_id_to_name = {0:'avocado',1:'banana'}

In [8]:
# By geaxgx
xml_body_1="""<annotation>
        <folder>FOLDER</folder>
        <filename>{FILENAME}</filename>
        <path>{PATH}</path>
        <source>
                <database>Unknown</database>
        </source>
        <size>
                <width>{WIDTH}</width>
                <height>{HEIGHT}</height>
                <depth>3</depth>
        </size>
"""
xml_object=""" <object>
                <name>{CLASS}</name>
                <pose>Unspecified</pose>
                <truncated>0</truncated>
                <difficult>0</difficult>
                <bndbox>
                        <xmin>{XMIN}</xmin>
                        <ymin>{YMIN}</ymin>
                        <xmax>{XMAX}</xmax>
                        <ymax>{YMAX}</ymax>
                </bndbox>
        </object>
"""
xml_body_2="""</annotation>        
"""

def save_to_voc(imgFile, xmlFile, aug, w, h):
    with open(xmlFile,"w") as f:
        f.write(xml_body_1.format(**{'FILENAME':os.path.basename(imgFile), 'PATH':imgFile,'WIDTH':w,'HEIGHT':h}))
        for i,bb in enumerate(aug['bboxes']):            
            f.write(xml_object.format(**{'CLASS':category_id_to_name[aug['category_id'][i]],'XMIN':int(aug['bboxes'][i][0]),'YMIN':int(aug['bboxes'][i][1]),'XMAX':int(aug['bboxes'][i][2]),'YMAX':int(aug['bboxes'][i][3])}))
        f.write(xml_body_2)

In [9]:
albumentationsError=0
datasetFolder=r'C:\Users\choowilson\Desktop\train'
for root, dirs, files in os.walk(datasetFolder):
    for file in files:
        if file.endswith(".jpg"):
            imageFile = osp.join(root,file)
            xmlFile = osp.join(root,file[:-4]+".xml")
            print(imageFile)
            print(xmlFile)
            print("")
            objects=[]
            classIds=[]
            objects , classIds = getObjects(xmlFile)
            image = cv2.imread(imageFile)
            annotations = {'image':image,'bboxes':objects,'category_id':classIds}
#             Generate how many augmented images for each image
            for i in range(2):
                try:
                    aug = get_aug(doc_aug)
                    augmented = aug(**annotations)
                    aug_h = augmented['image'].shape[0]
                    aug_w = augmented['image'].shape[1]
                    save_to_voc(f"{file[:-4]}_aug_{i}.jpg",f"{file[:-4]}_aug_{i}.xml",augmented,aug_w,aug_h)
                    cv2.imwrite(f"{file[:-4]}_aug_{i}.jpg",augmented['image'])
                except Exception as e:
                    print(e)
                    albumentationsError+=1

C:\Users\choowilson\Desktop\train\a1.jpg
C:\Users\choowilson\Desktop\train\a1.xml

C:\Users\choowilson\Desktop\train\a10.jpg
C:\Users\choowilson\Desktop\train\a10.xml

Requested crop size (200, 200) is larger than the image size (199, 300)
Requested crop size (200, 200) is larger than the image size (199, 300)
C:\Users\choowilson\Desktop\train\a11.jpg
C:\Users\choowilson\Desktop\train\a11.xml

C:\Users\choowilson\Desktop\train\a12.jpg
C:\Users\choowilson\Desktop\train\a12.xml

C:\Users\choowilson\Desktop\train\a13.jpg
C:\Users\choowilson\Desktop\train\a13.xml

Requested crop size (200, 200) is larger than the image size (176, 300)
C:\Users\choowilson\Desktop\train\a14.jpg
C:\Users\choowilson\Desktop\train\a14.xml

C:\Users\choowilson\Desktop\train\a15.jpg
C:\Users\choowilson\Desktop\train\a15.xml

C:\Users\choowilson\Desktop\train\a16.jpg
C:\Users\choowilson\Desktop\train\a16.xml

C:\Users\choowilson\Desktop\train\a17.jpg
C:\Users\choowilson\Desktop\train\a17.xml

C:\Users\choowilson\D

C:\Users\choowilson\Desktop\train\avcd72.jpg
C:\Users\choowilson\Desktop\train\avcd72.xml

C:\Users\choowilson\Desktop\train\avcd73.jpg
C:\Users\choowilson\Desktop\train\avcd73.xml

C:\Users\choowilson\Desktop\train\avcd8.jpg
C:\Users\choowilson\Desktop\train\avcd8.xml

C:\Users\choowilson\Desktop\train\avcd9.jpg
C:\Users\choowilson\Desktop\train\avcd9.xml

C:\Users\choowilson\Desktop\train\b1.jpg
C:\Users\choowilson\Desktop\train\b1.xml

C:\Users\choowilson\Desktop\train\b10.jpg
C:\Users\choowilson\Desktop\train\b10.xml

C:\Users\choowilson\Desktop\train\b11.jpg
C:\Users\choowilson\Desktop\train\b11.xml

C:\Users\choowilson\Desktop\train\b12.jpg
C:\Users\choowilson\Desktop\train\b12.xml

C:\Users\choowilson\Desktop\train\b13.jpg
C:\Users\choowilson\Desktop\train\b13.xml

C:\Users\choowilson\Desktop\train\b14.jpg
C:\Users\choowilson\Desktop\train\b14.xml

C:\Users\choowilson\Desktop\train\b15.jpg
C:\Users\choowilson\Desktop\train\b15.xml

C:\Users\choowilson\Desktop\train\b16.jpg
C:\Us

C:\Users\choowilson\Desktop\train\bnna23.jpg
C:\Users\choowilson\Desktop\train\bnna23.xml

C:\Users\choowilson\Desktop\train\bnna3.jpg
C:\Users\choowilson\Desktop\train\bnna3.xml

C:\Users\choowilson\Desktop\train\bnna4.jpg
C:\Users\choowilson\Desktop\train\bnna4.xml

C:\Users\choowilson\Desktop\train\bnna5.jpg
C:\Users\choowilson\Desktop\train\bnna5.xml

C:\Users\choowilson\Desktop\train\bnna6.jpg
C:\Users\choowilson\Desktop\train\bnna6.xml

C:\Users\choowilson\Desktop\train\bnna7.jpg
C:\Users\choowilson\Desktop\train\bnna7.xml

C:\Users\choowilson\Desktop\train\bnna8.jpg
C:\Users\choowilson\Desktop\train\bnna8.xml

C:\Users\choowilson\Desktop\train\bnna9.jpg
C:\Users\choowilson\Desktop\train\bnna9.xml

C:\Users\choowilson\Desktop\train\bnnn1.jpg
C:\Users\choowilson\Desktop\train\bnnn1.xml

C:\Users\choowilson\Desktop\train\bnnn2.jpg
C:\Users\choowilson\Desktop\train\bnnn2.xml

C:\Users\choowilson\Desktop\train\man-banana-24257122.jpg
C:\Users\choowilson\Desktop\train\man-banana-24257

In [10]:
print(albumentationsError)

6
